# Analyse CoViD-19 data

In [57]:
import pandas as pd
import numpy as np
from bqplot import *
import bqplot.pyplot as bqplt
from ipywidgets import Layout, Dropdown, Button
from ipywidgets import Image as ImageIpy
import matplotlib.colors as colors

In [58]:
shit =[[1,2,],[1,1] ]

In [59]:
colorListRGB =[ 
[                   0,                   0,   1.000000000000000],
[   1.000000000000000,                   0,                   0],
[                   0,   1.000000000000000,                   0],
[                   0,                   0,   0.172413793103448],
[   1.000000000000000,   0.103448275862069,   0.724137931034483],
[   1.000000000000000,   0.827586206896552,                   0],
[                   0,   0.344827586206897,                   0],
[   0.517241379310345,   0.517241379310345,   1.000000000000000],
[   0.620689655172414,   0.310344827586207,   0.275862068965517],
[                   0,   1.000000000000000,   0.758620689655172],
[                   0,   0.517241379310345,   0.586206896551724],
[                   0,                   0,   0.482758620689655],
[   0.586206896551724,   0.827586206896552,   0.310344827586207],
[   0.965517241379310,   0.620689655172414,   0.862068965517241],
[   0.827586206896552,   0.068965517241379,   1.000000000000000],
[   0.482758620689655,   0.103448275862069,   0.413793103448276],
[   0.965517241379310,   0.068965517241379,   0.379310344827586],
[   1.000000000000000,   0.758620689655172,   0.517241379310345],
[   0.137931034482759,   0.137931034482759,   0.034482758620690],
[   0.551724137931034,   0.655172413793103,   0.482758620689655],
[   0.965517241379310,   0.517241379310345,   0.034482758620690],
[   0.517241379310345,   0.448275862068966,                   0],
[   0.448275862068966,   0.965517241379310,   1.000000000000000],
[   0.620689655172414,   0.758620689655172,   1.000000000000000],
[   0.448275862068966,   0.379310344827586,   0.482758620689655]
]

In [64]:
colorListHTML = []
for color in colorListRGB:
    colorListHTML.append(colors.to_hex(color))

In [24]:
def getRawData(casesOrDeaths):
# Get URLs for data
    if casesOrDeaths.lower() ==  "cases":
        dataFile = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv"
    else:
        dataFile = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv"

    # Get data and clean up
    rawData = pd.read_csv(dataFile, index_col = 1).T
    rawData.drop(["Lat", "Long","Province/State"], inplace = True)
    rawData.drop(["Diamond Princess"], axis = "columns", inplace = True)

    # rawData.drop(["Cruise Ship", "Saint Lucia", "Taiwan*"], axis = "columns", inplace = True)
    rawData.rename(columns={"Iran": "Iran, Islamic Rep.", "Korea, South" : "Korea, Rep.", "US" : "United States", \
                        "Brunei": "Brunei Darussalam", "Czechia": "Czech Republic", "Egypt" : "Egypt, Arab Rep.", \
                        "Russia" : "Russian Federation", "Slovakia" : "Slovak Republic", "Congo (Kinshasa)" : "Congo, Dem. Rep.", \
                        "Guadeloupe" : "Mexico", "Jersey" : "United Kingdom", "Martinique" : "France", \
                        "Reunion" : "France", "Venezuela" : "Venezuela, RB"}, inplace = True)
    rawData = rawData.groupby(rawData.columns, axis=1).sum()
    endDate = pd.to_datetime(rawData.index[-1]).strftime("%d %B, %Y")
    rawData.reset_index(inplace = True, drop=True)
    return rawData

In [38]:
gAlign = 10
gMinimum = 15
gGradient = 'Cumulative'
gRollingMean = 0
gInterpolate = True
gRawData = getRawData("deaths")

In [39]:
def updatePlot(rawData, lineChart, align, minimum, gradient, rollingMean, interpolate):
    data = manipulateData(rawData, align, minimum, gradient, rollingMean, interpolate)
    lineChart.y = data
    lineChart.labels = list(data.index.to_numpy())

In [40]:
def manipulateData(rawData, align, minimum, gradient, rollingMean, interpolate):
    
    global gAlign
    global gMinimum
    global gGradient
    global gRollingMean
    global gInterpolate

    gAlign = align
    gMinimum = minimum
    gGradient = gradient
    gRollingMean = rollingMean
    gInterpolate = interpolate
    
    data = rawData.where(rawData > align)
    data = data.apply(lambda countryData: pd.Series(countryData.dropna().values))

    # Drop countries without minimum data
    data.dropna(axis = 'columns', thresh = minimum, inplace = True)

    # Interpolate
    if interpolate:
        data.where(data.diff() != 0, inplace = True)
        data.interpolate(method = "linear", limit = 2, limit_direction = "backward", inplace = True)

    # Take gradients
    if gradient == "New cases/deaths" or gradient == "Derivative of new cases/deaths":
        data = data.diff()

        # Rolling mean
        if rollingMean:
            data = data.rolling(rollingMean).mean()

        if gradient == "Derivative of new cases/deaths":
            data = data.diff()

#     # Truncate
#     data = data.truncate(after = cutoff)
    return data.T
# # Normalize
# for individualNormalizeDict in normalizeDicts:
#     data = data.apply(lambda countryData: countryData / individualNormalizeDict[countryData.name])


In [41]:
toggleCasesOrDeaths = widgets.ToggleButtons(
    options=['Deaths', 'Cases'],
    value="Deaths", 
#    layout={'width': 'max-content'}, # If the items' names are long
    description='Data:',
    disabled=False
)

In [42]:
sliderAlign = widgets.IntSlider(
    value=gAlign,
    min=1,
    max=100,
    step=1,
    description='Align:',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

In [43]:
sliderMinimum = widgets.IntSlider(
    value=gMinimum,
    min=1,
    max=len(gRawData),
    step=1,
    description='Minimum:',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

In [44]:
toggleGradient = widgets.ToggleButtons(
    options=['Cumulative', 'New cases/deaths', 'Derivative of new cases/deaths'],
    value=gGradient, 
#    layout={'width': 'max-content'}, # If the items' names are long
    description='Gradient:',
    disabled=False
)

In [45]:
sliderRollingMean = widgets.IntSlider(
    value=gRollingMean,
    min=0,
    max=15,
    step=1,
    description='Rolling Mean:',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

In [46]:
toggleInterpolate = widgets.ToggleButton(
    value=gInterpolate,
    description='Interpolate',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Description',
    icon='check' # (FontAwesome names without the `fa-` prefix)
)

In [47]:
toggleCasesOrDeaths.observe(lambda change : updatePlot(getRawData(change.new), lineChart, gAlign, gMinimum, gGradient, gRollingMean, gInterpolate), 'value')
sliderAlign.observe(lambda change : updatePlot(gRawData, lineChart, change.new, gMinimum, gGradient, gRollingMean, gInterpolate), 'value')
sliderMinimum.observe(lambda change : updatePlot(gRawData, lineChart, gAlign, change.new, gGradient, gRollingMean, gInterpolate), 'value')
toggleGradient.observe(lambda change : updatePlot(gRawData, lineChart, gAlign, gMinimum, change.new, gRollingMean, gInterpolate), 'value')
sliderRollingMean.observe(lambda change : updatePlot(gRawData, lineChart, gAlign, gMinimum, gGradient, change.new, gInterpolate), 'value')
toggleInterpolate.observe(lambda change : updatePlot(gRawData, lineChart, gAlign, gMinimum, gGradient, gRollingMean, change.new), 'value')

In [65]:
xData = gRawData.index.to_numpy()
yData = manipulateData(gRawData, gAlign, gMinimum, gGradient, gRollingMean, gInterpolate)

fig = bqplt.figure(animation_duration=500, legend_location = 'top-left')
lineChart = bqplt.plot(x=xData, y=yData, display_legend = True, labels = list(yData.index.to_numpy()), marker_str = 'sr', colors=colorListHTML)

In [66]:
sliderRollingMean.disabled = False

In [67]:
widgets.VBox([widgets.HBox([sliderAlign, toggleGradient ]), widgets.HBox([sliderMinimum, sliderRollingMean, toggleInterpolate]), toggleCasesOrDeaths, fig])

## To do:
- LineStyles
- Switch to OO method
- Posibility of log Y
- Normalise dicts
- Remove legend border
- Disable impossible combinations of toggles